<h2>Project 3: Na&iuml;ve Bayes</h2>

<blockquote>
    <center>
    <img src="nb.png" width="200px" />
    </center>
      <p><cite><center>"All models are wrong, but some are useful."<br>
       -- George E.P. Box
      </center></cite></p>
</blockquote>

<h3>Introduction</h3>
<!--Aðalbrandr-->

<p> You recently decided that you want to take the machine learning course at Cornell University. You're super excited to learn ML but are very confused during lectures. You realize that it's because your German professor just throws in German words while explaining concepts and you have no idea what these mean! You could just tell him you don't understand German but you're a proud engineer! There's got to be a better way. Something that doesn't involve communicating because, you know, that's not our forte (just kidding). So, you decide to create a system that detects every time a word is German and translates it for you in the subtitles. In this project, you will just implement the first part of this system using Na&iuml;ve Bayes to predict if a word is German or English. </p>
<p>
<strong>P3 Deadlines:</strong> 
The deadline for this project is on <strong>October 3 (11:59 pm EST)</strong>. The late deadline is on <strong>October 5</strong>.
</p>

<h3> English and German words </h3>

<p> Take a look at the files <code>english_train.txt</code> and <code>german_train.txt</code>. For example with the unix command <pre>cat german_train.txt</pre> 
<pre>
...
bibliothek
aufzuhalten
maegde
rupfen
leer
merkte
sucht
launenhaften
graeten
</pre>

The problem with the current file is that the words are in plain text, which makes it hard for a machine learning algorithm to learn anything useful from them. We therefore need to transform them into some vector format, where each word becomes a vector that represents a point in some high dimensional input space. </p>

<p>That is exactly what the following Python function <code>language2features</code> does: </p>

In [21]:
#<GRADED>
import numpy as np
import sys
from cvxpy import *
from matplotlib import pyplot as plt
#</GRADED>

%matplotlib inline


In [22]:
def feature_extraction_letters(word, B):
    v = np.zeros(B)
    for letter in word:
        v[ord(letter) - 97] += 1
    return v

In [23]:
def language2features(filename, B=26, LoadFile=True):
    """
    Output:
    X : n feature vectors of dimension B, (nxB)
    """
    if LoadFile:
        with open(filename, 'r') as f:
            words = [x.rstrip() for x in f.readlines() if len(x) > 0]
    else:
        words = filename.split('\n')
    n = len(words)
    X = np.zeros((n, B))
    for i in range(n):
        X[i,:] = feature_extraction_letters(words[i].lower(), B)
    return X

<p>It reads every word in the given file and converts it into a 26-dimensional feature vector by mapping each letter to a feature. The generated vector is a histogram containing the number of occurrences of each letter in the word.</p> 

<p>We have provided you with a python function <code>genFeatures</code>, which calls this function, transforms the words into features and loads them into memory. 

In [24]:
def genFeatures(dimension, language2features, file_german, file_english):
    """
    function [x,y]=genFeatures
    
    This function calls "language2features.py" to convert 
    words into feature vectors and load training data. 
    
    language2features: function that extracts features from language word
    dimension: dimensionality of the features
    
    Output: 
    x: n feature vectors of dimensionality d [n,d]
    y: n labels (-1 = German, +1 = English)
    """
    
    # Load in the data
    Xgerman = language2features(file_german, B=dimension)
    Xenglish = language2features(file_english, B=dimension)
    X = np.concatenate([Xgerman, Xenglish])
    
    # Generate Labels
    Y = np.concatenate([-np.ones(len(Xgerman)), np.ones(len(Xenglish))])
    
    # shuffle data into random order
    ii = np.random.permutation([i for i in range(len(Y))])
    
    return X[ii, :], Y[ii]

You can call the following command to load features and labels of all German and English words.

In [25]:
X,Y = genFeatures(26, language2features, "german_train.txt", "english_train.txt")
xTe, yTe = genFeatures(26, language2features, "german_test.txt", "english_test.txt")

<h3> Multinomial Na&iuml;ve Bayes Classifier </h3>

<p> The Na&iuml;ve Bayes classifier is a linear classifier based on Bayes Rule. The following questions will ask you to finish these functions in a pre-defined order. <br></p>
<p>(a) Estimate the class probability P(Y) in 
<b><code>naivebayesPY</code></b>
. This should return the probability that a sample in the training set is positive or negative, independent of its features.
</p>



In [26]:
#<GRADED>
def naivebayesPY(x,y):
    """
    function [pos,neg] = naivebayesPY(x,y);

    Computation of P(Y)
    Input:
        x : n input vectors of d dimensions (n,d)
        y : n labels (-1 or +1) (n,)

    Output:
    pos: probability p(y=1)
    neg: probability p(y=-1)
    """
    
    
    pos_list = [1 if ys == 1 else 0 for ys in y]
    pos = np.sum(pos_list)/len(y)
    neg_list = [1 if ys == -1 else 0 for ys in y]
    neg = np.sum(neg_list)/len(y)
    
    return pos, neg

#</GRADED>

pos,neg = naivebayesPY(X,Y)
neg

0.5

<p>(b) Estimate the conditional probabilities P(X|Y) <b>(Maximum Likelihood Estimate)</b> without smoothing in 
<b><code>naivebayesPXY_mle</code></b>
.  Use a <b>multinomial</b> distribution as model. This will return the probability vectors  for all features given a class label.
Stated differently, the output vector should have dimensions equal to the number of letters of the alphabet. For each index i, we ask you to find the probability that any letter of any word contains this ith letter of the alphabet.  
</p> 

In [27]:
#<GRADED>
def naivebayesPXY_mle(x,y):
    """
    function [posprob,negprob] = naivebayesPXY(x,y);
    
    Computation of P(X|Y) -- Maximum Likelihood Estimate
    Input:
        x : n input vectors of d dimensions (n,d)
        y : n labels (-1 or +1) (n,)
    
     Output:
    posprob: probability vector of p(x=ith letter of alphabet|y=1) (d,)
    negprob: probability vector of p(x=ith letter of alphabet|y=-1) (d,)
    """
    
    pos_indices = np.where(y == 1)[0]
    neg_indices = np.where(y == -1)[0]
    
    n, d = x.shape
    posprob = np.zeros(d)
    negprob = np.zeros(d)
    
    for i in range(d):
        pos_count = np.sum(x[pos_indices, i])
        neg_count = np.sum(x[neg_indices, i])
        
        total_pos_count = np.sum(x[pos_indices])
        total_neg_count = np.sum(x[neg_indices])
        
        posprob[i] = (pos_count) / (total_pos_count)  
        negprob[i] = (neg_count) / (total_neg_count)  
    
    return posprob.flatten(), negprob.flatten()
    
    
#</GRADED>

posprob_mle,negprob_mle = naivebayesPXY_mle(X,Y)
posprob_mle

array([0.08078045, 0.01732312, 0.04868709, 0.03501094, 0.11579139,
       0.01495259, 0.01914661, 0.02297593, 0.08698031, 0.00182349,
       0.00948213, 0.05634573, 0.02881109, 0.07239241, 0.06363968,
       0.03081692, 0.00164114, 0.07640408, 0.0607221 , 0.07749818,
       0.03136397, 0.01331145, 0.01002918, 0.00273523, 0.02005835,
       0.00127644])

In [28]:
np.sum(X[3])

7.0

<p>(c) Estimate the conditional probabilities P(X|Y) <b>(Smoothing with Laplace estimate)</b> in 
<b><code>naivebayesPXY_smoothing</code></b>
.  Use a <b>multinomial</b> distribution as model. This will return the probability vectors  for all features given a class label.
</p> 

In [29]:
#<GRADED>
def naivebayesPXY_smoothing(x,y):
    """
    function [posprob,negprob] = naivebayesPXY(x,y);
    
    Computation of P(X|Y) -- Smoothing with Laplace estimate
    Input:
        x : n input vectors of d dimensions (n,d)
        y : n labels (-1 or +1) (n,)
    
    Output:
    posprob: probability vector of p(x|y=1) (d,)
    negprob: probability vector of p(x|y=-1) (d,)
    """
  
    n, d = x.shape
    
    pos_indices = np.where(y == 1)[0]
    neg_indices = np.where(y == -1)[0]
    
    posprob = np.ones(d)  
    negprob = np.ones(d)  
    
    for i in range(d):
        pos_count = np.sum(x[pos_indices, i])
        neg_count = np.sum(x[neg_indices, i])
        
        total_pos_count = np.sum(x[pos_indices])
        total_neg_count = np.sum(x[neg_indices])
        
        posprob[i] = (pos_count + 1) / (total_pos_count + d)  
        negprob[i] = (neg_count + 1) / (total_neg_count + d)  
    
    return posprob, negprob
    
    
    
    
#</GRADED>

posprob_smoothing,negprob_smoothing = naivebayesPXY_smoothing(X,Y)
posprob_smoothing

array([0.08058076, 0.01742287, 0.04863884, 0.03502722, 0.1154265 ,
       0.01506352, 0.01923775, 0.023049  , 0.08675136, 0.00199637,
       0.00961887, 0.05626134, 0.02885662, 0.0722323 , 0.06352087,
       0.03085299, 0.00181488, 0.07622505, 0.06061706, 0.07731397,
       0.03139746, 0.01343013, 0.01016334, 0.00290381, 0.02014519,
       0.00145191])

In [30]:
X[1,:]

array([0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 0., 0.])

## <p>(d) Solve for the log ratio, $\log\left(\frac{P(Y=1 | X = xtest)}{P(Y=-1|X= xtest)}\right)$, using Bayes Rule. Doing this in log space is important to avoid underflow. Make sure to first log the probabilities and then sum them.
 Implement this in 
<b><code>naivebayes</code></b>.
</p>



In [31]:
import math
#<GRADED>
def naivebayes(x,y,xtest,naivebayesPXY):
    """
    function logratio = naivebayes(x,y);
    
    Computation of log P(Y|X=x1) using Bayes Rule
    Input:
    x : n input vectors of d dimensions (n,d)
    y : n labels (-1 or +1)(n,)
    xtest: input vector of d dimensions (d,)
    naivebayesPXY: input function for getting conditional probabilities (naivebayesPXY_smoothing OR naivebayesPXY_mle)
    
    Output:
    logratio: log (P(Y = 1|X=xtest)/P(Y=-1|X=xtest))
    """
    
    posprob, negprob = naivebayesPXY(x, y)
    
    pos_list = [1 if ys == 1 else 0 for ys in y]
    prior_pos = np.sum(pos_list)/len(y)
    neg_list = [1 if ys == -1 else 0 for ys in y]
    prior_neg = np.sum(neg_list)/len(y)
    

    pos_log = 1.0
    neg_log = 1.0
    for i in range(len(xtest)):
        pos_log *= (posprob[i]**xtest[i])
        neg_log *= (negprob[i]**xtest[i])
    
    CP_pos = np.log(pos_log) + np.log(prior_pos)
    CP_neg = np.log(neg_log) + np.log(prior_neg)
    logratio = (CP_pos - CP_neg)
    
    
    return logratio
    
#</GRADED>
p_smoothing = naivebayes(X,Y,X[0,:], naivebayesPXY_smoothing)
p_mle = naivebayes(X,Y,X[0,:], naivebayesPXY_mle)

<p>(e) Naïve Bayes can also be written as a linear classifier. Implement this in 
<b><code>naivebayesCL</code></b>.

In [32]:
#<GRADED>
def naivebayesCL(x,y,naivebayesPXY):
    """
    function [w,b]=naivebayesCL(x,y);
    Implementation of a Naive Bayes classifier
    Input:
    x : n input vectors of d dimensions (n,d)
    y : n labels (-1 or +1)(n,)
    naivebayesPXY: input function for getting conditional probabilities (naivebayesPXY_smoothing OR naivebayesPXY_mle)

    Output:
    w : weight vector of d dimensions (d,)
    b : bias (scalar)
    """
    
    n, d = x.shape
    posprob, negprob = naivebayesPXY(x, y)
    
    pos_list = [1 if ys == 1 else 0 for ys in y]
    prior_pos = np.sum(pos_list)/len(y)
    neg_list = [1 if ys == -1 else 0 for ys in y]
    prior_neg = np.sum(neg_list)/len(y)
    
    w = np.log(posprob / negprob)
    
    
    b = np.log(prior_pos/prior_neg)
    return w, b


#</GRADED>

w_smoothing,b_smoothing = naivebayesCL(X,Y, naivebayesPXY_smoothing)
w_mle,b_mle = naivebayesCL(X,Y, naivebayesPXY_mle)

<p>(f) Implement 
<b><code>classifyLinear</code></b>
 that applies a linear weight vector and bias to a set of input vectors and outputs their predictions.  (You can use your answer from the previous project.)
 
 

In [33]:
#<GRADED>
def classifyLinear(x,w,b=0):
    """
    function preds=classifyLinear(x,w,b);
    
    Make predictions with a linear classifier
    Input:
    x : n input vectors of d dimensions (n,d)
    w : weight vector of d dimensions (d,)
    b : bias (optional)
    
    Output:
    preds: predictions
    """
    
    w = w.flatten()    
    n,_ = x.shape
    preds = np.zeros(n)
    preds += np.dot(x,w) + b
    preds = np.sign(preds)  
    return preds

#</GRADED>

print('Training error (Smoothing with Laplace estimate): %.2f%%' % (100 *(classifyLinear(X, w_smoothing, b_smoothing) != Y).mean()))
print('Training error (Maximum Likelihood Estimate): %.2f%%' % (100 *(classifyLinear(X, w_mle, b_mle) != Y).mean()))
print('Test error (Smoothing with Laplace estimate): %.2f%%' % (100 *(classifyLinear(xTe, w_smoothing, b_smoothing) != yTe).mean()))
print('Test error (Maximum Likelihood Estimate): %.2f%%' % (100 *(classifyLinear(xTe, w_mle, b_mle) != yTe).mean()))


Training error (Smoothing with Laplace estimate): 25.56%
Training error (Maximum Likelihood Estimate): 25.44%
Test error (Smoothing with Laplace estimate): 25.67%
Test error (Maximum Likelihood Estimate): 25.50%


You can now test your code with the following interactive word classification script. Use the word 'exit' to stop the program.

In [34]:
DIMS = 26
print('Loading data ...')
X,Y = genFeatures(DIMS, language2features, "german_train.txt", "english_train.txt")
xTe, yTe = genFeatures(26, language2features, "german_test.txt", "english_test.txt")
print('Training classifier (Smoothing with Laplace estimate) ...')
w,b=naivebayesCL(X,Y,naivebayesPXY_smoothing)
train_error = np.mean(classifyLinear(X,w,b) != Y)
test_error = np.mean(classifyLinear(xTe,w,b) != yTe)
print('Training error (Smoothing with Laplace estimate): %.2f%%' % (100 * train_error))
print('Test error (Smoothing with Laplace estimate): %.2f%%' % (100 * test_error))

yourword = ""
while yourword!="exit":
    yourword = input()
    if len(yourword) < 1:
        break
    xtest = language2features(yourword,B=DIMS,LoadFile=False)
    pred = classifyLinear(xtest,w,b)[0]
    if pred > 0:
        print("%s, is an english word.\n" % yourword)
    else:
        print("%s, is a german word.\n" % yourword)

Loading data ...
Training classifier (Smoothing with Laplace estimate) ...
Training error (Smoothing with Laplace estimate): 25.56%
Test error (Smoothing with Laplace estimate): 25.67%
exit
exit, is an english word.



<h3> Feature Extraction</h3>

<p> Similar to how we extracted features from words in <code>feature_extraction_letters</code>, we are going to try another way of doing so. This time, instead of mapping a letter to a feature, we will map a pair of letters to a feature. </p>
    
<p>
Every element in the feature vector will represent a pair of letters (e.g. 'aa', 'ab', 'ac'...) and the element representing the pair of letters that occur in the word will be the number of occurence. Make sure your feature vector <b> ordering is alphabetical </b> i.e. ['aa', 'ab', 'ac'.....'ba', 'bb'......'ca','cb'......]. The length of the feature vector will be $26^2 = 676$ to represent all possible pairs of 26 letters. Assume everything is in lower case.
</p>

<p>
Here's an example, for the word 'mama', elements in the feature vector representing 'ma' will be 2, 'am' will be 1. All the other 674 features will be 0.
</p>

<p>
Please modify <code><b>feature_extraction_letters_pairs</b></code> below to implement this feature extraction.
</p>

In [40]:
#<GRADED>
def feature_extraction_letters_pairs(word, B=676):
    """
    Feature extration from word for pairs
    word: word of the language as a string
    
    Output:
    v : a feature vectors of dimension B=676, (B,)
    """
    v = np.zeros(B)
    
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    alphabet_pairs = []
    for i in range(len(alphabet)):
        for j in range(len(alphabet)):
            pair = alphabet[i] + alphabet[j]
            alphabet_pairs.append(pair)
    
    for j in range(len(word)-1):
        idx = alphabet_pairs.index(word[j]+word[j+1])
        v[idx] += 1


    return v
    
def language2features_pairs(filename, B=676, LoadFile=True):
    """
    Output:
    X : n feature vectors of dimension B, (n,B)
    """
    if LoadFile:
        with open(filename, 'r') as f:
            words = [x.rstrip() for x in f.readlines() if len(x) > 0]
    else:
        words = filename.split('\n')
    n = len(words)
    X = np.zeros((n, B))
    for i in range(n):
        X[i,:] = feature_extraction_letters_pairs(words[i].lower(), B)
    return X

#</GRADED>

In [41]:
feature_extraction_letters_pairs('mama', B=676)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [18]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'
alphabet_pairs = []
for i in range(len(alphabet)):
    for j in range(len(alphabet)):
        pair = alphabet[i] + alphabet[j]
        alphabet_pairs.append(pair)

len(alphabet_pairs)

676

You can now test your code with the following interactive word classification script:

In [42]:
''' result of the Naive Bayes classifier using pairs of letters as features '''
DIMS = 676
print('Loading data ...')
Xp,Yp = genFeatures(676, language2features_pairs, "german_train.txt", "english_train.txt")
xTe, yTe = genFeatures(676, language2features_pairs, "german_test.txt", "english_test.txt")
print('Training classifier (Smoothing with Laplace estimate) ...')
w,b=naivebayesCL(Xp,Yp,naivebayesPXY_smoothing)
train_error = np.mean(classifyLinear(Xp,w,b) != Yp)
print('Training error (Smoothing with Laplace estimate): %.2f%%' % (100 * train_error))
test_error = np.mean(classifyLinear(xTe,w,b) != yTe)
print('Test error (Smoothing with Laplace estimate): %.2f%%' % (100 * test_error))

yourword = ""
while yourword!="exit":
    yourword = input()
    if len(yourword) < 1:
        break
    xtest = language2features(yourword,B=DIMS,LoadFile=False)
    pred = classifyLinear(xtest,w,b)[0]
    if pred > 0:
        print("%s, is an english word.\n" % yourword)
    else:
        print("%s, is a german word.\n" % yourword)

Loading data ...
Training classifier (Smoothing with Laplace estimate) ...
Training error (Smoothing with Laplace estimate): 11.25%
Test error (Smoothing with Laplace estimate): 14.75%
exit
exit, is an english word.



Why do you think we are not running the naïve Bayes classifier with the __maximum likelihood estimates__ in the previous cell?